In [1]:
import os
import sys
import warnings
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse

In [3]:
import logging
logging.basicConfig(level=logging.WARN)


In [4]:
logger = logging.getLogger(__name__)
csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
train, test = train_test_split(data,test_size= 0.8)
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [7]:
def train_model(train_x, train_y, alpha = 0.5, l1_ratio = 0.5):
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
    return lr, alpha, l1_ratio

In [8]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [9]:
import mlflow 
import mlflow.sklearn

In [10]:
with mlflow.start_run():
    lr, alpha, l1_ratio = train_model(train_x, train_y)
    predicted_qualities = lr.predict(test_x) 
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)    
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)    
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)    
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
    else:
        mlflow.sklearn.log_model(lr, "model")
        
        
        
        
        
        
        
        
        
        

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7352657423688538
  MAE: 0.5948151285161909
  R2: 0.16111213803874602


In [11]:
#with mlflow.start_run()

In [12]:
with mlflow.start_run():
    lr, alpha, l1_ratio = train_model(train_x, train_y, 0.8, 0.9)
    predicted_qualities = lr.predict(test_x) 
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)    
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)    
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)    
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
    else:
        mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.800000, l1_ratio=0.900000):
  RMSE: 0.7894353159362162
  MAE: 0.6550094901305428
  R2: 0.03295129202398961


In [13]:
with mlflow.start_run():
    lr, alpha, l1_ratio = train_model(train_x, train_y, 0.7, 0.8)
    predicted_qualities = lr.predict(test_x) 
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)    
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)    
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)    
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
    else:
        mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.700000, l1_ratio=0.800000):
  RMSE: 0.7891551359371908
  MAE: 0.6539090617793474
  R2: 0.03363760442807173


In [ ]:
!mlflow ui

[2021-08-19 22:19:45 +0200] [3533] [INFO] Starting gunicorn 20.1.0
[2021-08-19 22:19:45 +0200] [3533] [INFO] Listening at: http://127.0.0.1:5000 (3533)
[2021-08-19 22:19:45 +0200] [3533] [INFO] Using worker: sync
[2021-08-19 22:19:45 +0200] [3536] [INFO] Booting worker with pid: 3536
